In [3]:
import pandas as pd

# Read the problem data from a CSV file
data = pd.read_csv('transportation.csv', index_col=0)
data

,A,B,C,D,Supply
O1,19,30,50,10,7.0
O2,70,30,40,60,9.0
O3,40,8,70,20,18.0
Demand,5,8,70,20,NaN


In [ ]:

# Get the list of rows and columns
rows = list(data.index)
cols = list(data.columns)

# Get the supply and demand lists
supply = list(data['Supply'])
demand = list(data.loc['Demand'])

# Initialize the allocations, row and column penalties
allocations = pd.DataFrame(0, index=rows, columns=cols)
u = [0] * len(rows)
v = [0] * len(cols)

# Calculate the row and column penalties
for i in range(len(rows)):
    diff = sorted(data.loc[rows[i]][data.loc[rows[i]] > 0])[0:2]
    u[i] = diff[1] - diff[0]

for j in range(len(cols)):
    diff = sorted(data[cols[j]][data[cols[j]] > 0])[0:2]
    v[j] = diff[1] - diff[0]

# Perform the allocations using Vogel's approximation method
while sum(supply) > 0 and sum(demand) > 0:
    # Find the row and column with the largest penalties
    i = u.index(max(u))
    j = v.index(max(v))

    # Allocate as much as possible to the cell with the largest penalty
    if u[i] > v[j]:
        allocations.loc[rows[i], cols[j]] = demand[j]
        supply[i] -= demand[j]
        demand[j] = 0
        v[j] = -1
        u[i] = -1
    else:
        allocations.loc[rows[i], cols[j]] = supply[i]
        demand[j] -= supply[i]
        supply[i] = 0
        u[i] = -1
        v[j] = -1

    # Update the row and column penalties
    for i in range(len(rows)):
        if supply[i] > 0:
            diff = sorted(data.loc[rows[i]][data.loc[rows[i]] > allocations.loc[rows[i]].sum()])[0:2]
            u[i] = diff[1] - diff[0]
        else:
            u[i] = -1

    for j in range(len(cols)):
        if demand[j] > 0:
            diff = sorted(data[cols[j]][data[cols[j]] > allocations[cols[j]].sum()])[0:2]
            v[j] = diff[1] - diff[0]
        else:
            v[j] = -1

# Calculate the total cost of the allocations
total_cost = (allocations * data).sum().sum()

# Print the allocations and total cost
print("Allocations:")
print(allocations)
print("Total cost:", total_cost)
